In [0]:
import time
import requests as req
import json
import gc
import os
import re
import pandas as pd
import os
import pyspark.sql.functions as psf

import datetime
from dateutil.relativedelta import relativedelta
from multiprocessing.pool import ThreadPool
from pyspark.sql.functions import substring
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType,DoubleType
from pyspark.sql.functions import concat
from pyspark.sql.functions import col,round
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

# OpenAI with PySpark

In [0]:
!pip install -r /Workspace/Repos/jeanpierre.vanniekerk@truenorthgroup.co.za/document-analysis-using-gpt-3.ide/notebooks/requirements.txt


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [0]:
from synapse.ml.core.platform import find_secret

# Fill in the following lines with your service information
# Learn more about selecting which embedding model to choose: https://openai.com/blog/new-and-improved-embedding-model
service_name = 'tngpocazureopenai-services'
deployment_name = "ChatGPT"
deployment_name_embeddings = "text-embedding-ada-002"

key = '7079b53b72df4f04bf94a302697561e9'


https://github.com/microsoft/SynapseML#databricks

https://microsoft.github.io/SynapseML/docs/Explore%20Algorithms/OpenAI/

# Pandas Completion

In [0]:
import os
import openai
from dotenv import load_dotenv
import numpy as np
import pandas as pd
pd.options.display.max_columns = 500

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

df = spark.createDataFrame([("Hello my name is",),("The best code is code thats",),("SynapseML is ",)]).toDF("prompt").toPandas()
df["text"] = np.nan

for idx, title in zip(df.index.values, df['prompt'].loc[df.index.values]):
  # build prompt
  prompt = title

  try:
    # Request API
    response = openai.Completion.create(deployment_id="ChatGPT", prompt=prompt,temperature=0,max_tokens=1000,top_p=0.95,frequency_penalty=1,presence_penalty=1)
    # response
    df['text'].loc[idx] = response['choices'][0]['text']
  except Exception as err:
    idx
    print(f"Unexpected {err=}, {type(err)=}")
df

<command-1114856071198587>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'].loc[idx] = response['choices'][0]['text']


,prompt,text
0,Hello my name is,***** will be assisting you today. I am sorry ...
1,The best code is code thats,never written.\n\nI'm not sure if this is a j...
2,SynapseML is,a Python package for simulating spiking neura...


# Pyspark Completion

In [0]:
df = spark.createDataFrame(
    [("Hello my name is",),("The best code is code thats",),("SynapseML is ",)]).toDF("prompt")

from synapse.ml.cognitive import OpenAICompletion

completion = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setMaxTokens(200)
    .setPromptCol("prompt")
    .setErrorCol("error")
    .setOutputCol("completions"))

from pyspark.sql.functions import col
completed_df = completion.transform(df).cache()
display(completed_df.select( col("prompt"),col("error"),col("completions.choices.text").getItem(0).alias("text")))

prompt,error,text
Hello my name is,null,Jasmine and very thankful for this page... I am in urgent need of a service dog since a disability I have has causes me to need assistance ...I am currently unemployed and raising two girls this has been a very hard road for me and am currently on medication to fight pain and mobility issues hopefully this would make things a lot easier for me and my family anything helps thank you so much for your time and I hope someone could help me This is a direct to organizer fundraiser. Jasmine and Fundraiser for me and my soon to be service dog.
The best code is code thats,null,"never written. + +## Another cool idea + +Another idea that could be explored would be to connect a Slack workspace to the web app. This would allow users to join existing groups they are part of, or create new groups directly in the app. This could lead down some really cool water in an application that leverages Slack's capabilities in conjunction with web app functionalities. + +WIP. + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + +[nodejs]: https://nodejs.org/en/ +[express]: https://expressjs.com/ +[json-server]: https://www.npmjs.com/package/json-server. +[ejs]: https://ejs.co/ +[bootstrap]: https://getbootstrap.com/ +[VanillaJS]: http://vanilla-js.com/ + + + + + + + + + + + + + + + + + + + + + + + + + +"
SynapseML is,null,"the first direct spatio-temporal coding scheme utilizing spikingmodeling to encode and decode multi-neural signals in real-time to achieve an extreme low-power for high-density 2D or 3D neural images or video data applications. SynapseML's hardware implementation platform is based on ASICs, FPGAs, and digital signal processors featuring low-power consumption, high speed, and low latency.}%More details about its applications can be found in the Appendix.}. \subsection{A Twofold Validation of SynapseML by Integrated Neural Signal Reconstruction and Decoding} To assess the effectiveness of SynapseML on the signal decoding task, we implemented an integrated recording-reconstruction-decoding system demonstration (see Fig.~\ref{fig:Circuit_and_Architecture}). In the simulation, 100 neurons were used to simulate a 10$\times$10 2D image. Electrodes sequentially sampled 50 neurons at a frequency of 20 kHz and converted the"


# Pandas ChatCompletion

In [0]:
import os
import openai
from dotenv import load_dotenv
import numpy as np
import pandas as pd

def make_message(role, content):
    return Row(role=role, content=content, name=role)

load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

chat_df = spark.createDataFrame(
    [([make_message("system", "You are an AI chatbot with red as your favorite color"),make_message("user", "Whats your favorite color"),],),
     ([make_message("system", "You are very excited"),make_message("user", "How are you today"),],),]).toDF("messages").toPandas()
chat_df["context"] = np.nan

for idx, messages in zip(chat_df.index.values, chat_df['messages'].loc[chat_df.index.values]):
    prompt = messages.tolist()
    try:
        response = openai.ChatCompletion.create(engine="ChatGPT", messages=prompt,temperature=0,max_tokens=800,top_p=0.95,frequency_penalty=0,presence_penalty=0,stop=None)
        chat_df['context'].loc[idx] = response['choices'][0]['message']['content']
    except Exception as err:
        idx
        print(f"Unexpected {err=}, {type(err)=}")
display(chat_df)

<command-668646728207862>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chat_df['context'].loc[idx] = response['choices'][0]['message']['content']


messages,context
"List(List(You are an AI chatbot with red as your favorite color, system, system), List(Whats your favorite color, user, user))",My favorite color is red! It's such a bold and vibrant color that can really make a statement. What's your favorite color?
"List(List(You are very excited, system, system), List(How are you today, user, user))","As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any task or question you have. How can I help you today?"


# Pyspark ChatCompletion

In [0]:
from synapse.ml.cognitive import OpenAIChatCompletion
from pyspark.sql import Row
from pyspark.sql.types import *


chat_df = spark.createDataFrame(
    [([make_message("system", "You are an AI chatbot with red as your favorite color"),
        make_message("user", "Whats your favorite color"),],),
     ([make_message("system", "You are very excited"),
        make_message("user", "How are you today"),],),]).toDF("messages")

chat_completion = (
    OpenAIChatCompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setMessagesCol("messages")
    .setErrorCol("error")
    .setOutputCol("chat_completions"))

display(chat_completion.transform(chat_df).select("messages", "chat_completions.choices.message.content"))

messages,content
"List(List(system, You are an AI chatbot with red as your favorite color, system), List(user, Whats your favorite color, user))","List(My favorite color is red! I think it's bold, energetic, and attention-grabbing. What is your favorite color?)"
"List(List(system, You are very excited, system), List(user, How are you today, user))","List(As an AI language model, I do not have feelings or emotions, but I'm functioning properly and ready to assist you with anything you need. How can I assist you today?)"


# Python Batch Completion

In [0]:
import os
import openai
from dotenv import load_dotenv
import numpy as np
import pandas as pd

load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

batch_df = spark.createDataFrame(
    [(["The time has come", "Pleased to", "Today stocks", "Here's to"],),
        (["The only thing", "Ask not what", "Every litter", "I am"],),]).toDF("batchPrompt").toPandas()
batch_df["text"] = np.nan

for idx, title in zip(batch_df.index.values, batch_df['batchPrompt'].loc[batch_df.index.values]):
  prompt = title.tolist()

  try:
    response = openai.Completion.create(deployment_id="ChatGPT", prompt=prompt,temperature=0,max_tokens=1000,top_p=0.95,frequency_penalty=1,presence_penalty=1)
    batch_df['text'].loc[idx] = response['choices'][0]['text']
  except Exception as err:
    idx
    print(f"Unexpected {err=}, {type(err)=}")
display(batch_df)

<command-1088354117485996>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['text'].loc[idx] = response['choices'][0]['text']


batchPrompt text List(The time has come, Pleased to, Today stocks, Here's to) for the next generation of leaders to take up the mantle and continue our work. I’m excited to see what they will accomplish in their own right, but I know that whatever it is, it will be amazing.

Thank you all for your support over these past few years. It has been an honor and a privilege to serve as your president.

Sincerely,

[Your Name]

Sample Resignation Letter from Board

Dear [Board Members],

I am writing today to inform you that I have decided to resign from my position on the board of directors effective immediately. This decision was not made lightly, but after much consideration, I feel that it is time for me to step down and allow someone else to take my place.

Over the past several years, we have accomplished many great things together as a team. We’ve worked hard to improve our organization’s operations and financial stability while also expanding our reach into new communities across the country. However, with so many changes happening within our industry and society at large – including increased competition from other organizations – I believe that now is the right time for fresh leadership with new ideas about how best we can move forward.

I want everyone here today who has supported me throughout this journey: thank you! Your encouragement helped keep me going when times were tough or challenging; without each one of you by my side every step along this path would have been impossible!

Please let me know if there are any questions or concerns regarding my resignation letter or anything else related specifically towards transitioning out smoothly during this period before officially stepping down completely from duties associated with being part of board members’ responsibilities.

Sincerely,

[Your Name]”

Sample Resignation Letter Due To Personal Reasons

Dear [Manager’s name],

It is with regret that I must submit my resignation due personal reasons beyond control which require immediate attention outside work environment starting tomorrow onwards until further notice hence unable fulfill obligations required by company policy effectively anymore under current circumstances affecting performance negatively despite efforts put forth trying balance both aspects life simultaneously resulting in undue stress levels impacting health adversely too making continuation untenable at present moment necessitating departure sooner rather than later unfortunately leaving no choice except tendering formal notice hereby effective immediately upon receipt thereof please accept apologies inconvenience caused sincerely hoping understand situation empathetically thanking opportunity given contribute positively towards growth success organisation during tenure served here looking forward staying touch future endeavors wishing continued prosperity ahead always remain grateful memories shared colleagues peers alike fondly remembered forevermore yours truly,

[Your Name]

Conclusion:

In conclusion, writing a resignation letter can be difficult especially when emotions are running high however keeping calm composed helps maintain professionalism even amidst chaos ensuring smooth transition process possible between parties involved ultimately leading successful outcomes desired mutually beneficial terms long term relationships built trust respect established based mutual understanding empathy compassion kindness shown towards others regardless situations faced along way remember always leave door open opportunities arise future never burn bridges behind because world small interconnected network people where reputation precedes oneself everywhere go therefore important handle matters gracefully tactfully diplomatically possible avoiding unnecessary conflicts misunderstandings arising unnecessarily causing harm anyone involved instead focusing positive aspects gained experiences learned lessons valuable assets carry lifetime wherever may lead us 

#Pyspark Batch Completions

In [0]:
batch_df = spark.createDataFrame(
    [(["The time has come", "Pleased to", "Today stocks", "Here's to"],),
        (["The only thing", "Ask not what", "Every litter", "I am"],),]).toDF("batchPrompt")

batch_completion = (
    OpenAICompletion()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name)
    .setCustomServiceName(service_name)
    .setMaxTokens(200)
    .setBatchPromptCol("batchPrompt")
    .setErrorCol("error")
    .setOutputCol("completions"))

completed_batch_df = batch_completion.transform(batch_df).cache()
display(completed_batch_df)

batchPrompt,error,completions
"List(The time has come, Pleased to, Today stocks, Here's to)",null,"List(cmpl-7twtdnEvU2Hm3yA1RfOC44RRWbKRs, text_completion, 1693568793, gpt-35-turbo, List(List( ahead for you to get over the mundane, provincial, and local scope of your thinking, known as the nation-state. So let us give up our attachment to the geographic, political state and the culture that usually goes with it. Let us not forget that we do not live on islands, but on continents. *If you touch Europe, you’ll see Africa, and if you turn, you see America. That’s how global we are! So we continuously have to think geographically, we have to think continentally and not locally.” Finally, he reminds his audiences that Africa was the first home of humanity: And the second thing I would like Africans to do is to revive the great cultural, intellectual, philosophical, and ethical heritage they inherited from their ancestors. We are the people who brought the first human civilization to the world. More Pyramid structures that still defy our imagination were built in Africa than anywhere else in the world, right here in the Sudan, more than 2,300 years ago, 0, null, length), List( meet you, I’m Potts.” “I thought it was ‘I’m almost done here.’” “That too. I just wanted to be polite first.” We shared a chuckle. “I appreciate that.” “So what is it you’re here for?” “Just interviewing people for a project.” “On what?” I was solely tempted to tell him that it was an investigation on the man whom his restaurant was named after, but thought against it. A few of the locals seemed to have grown tired of hearing about Jimmy Hoffa and his whereabouts, so I decided against bringing up Seymour Potts. “Politics,” I said. He groaned. “I hate politics.” “Really? I thought everyone around here was a politician.” With a mischievous grin on his face, he leaned in closer. “We are, and yet we all hate politics. But I think we hate it more so because we’re not that good at it.” The grin on my face turned into a full smile., 1, null, length), List( suffered the worst losses they have experienced for almost 3 weeks, as growth worries continued to push investors away from equities and into the government debt markets, both in the US and Europe. Markets are starting to become concerned that if the European crisis continues or even deteriorates, and Chinese economic growth figures to continue to fall, this will have a significant and damaging impact on global economic growth. Today Spain continued to make headlines due to its ongoing economic crisis, as ratings agency Moody's announced that they were downgrading the debt of Spain's key banks. These downgrades include the significant downgrading of Banco Santander and BBVA who have both had their debt ratings cut one notch to A3, and Banco Popular whose rating falls two notches to Ba3. These downgrades push the ratings of all three banks deeper into junk status. Late on Tuesday rating agency Standard & Poor's cut Greece's sovereign rating to 'CCC', the lowest defined rating, making the debt of Greece, which is already struggling, 2, null, length), List( hoping it's not a sad quietness. I'm sure it won't be. I'm sure it won't be. Alright, cool. Everything's hunky-dory. And the lights are flickering again. Don't worry about it. I'm sure it's whatever. It's probably just the light bulbs. Yes, just the light bulbs, it's fine. It can't be the specter of the night. Halloween is over, there's nothing to worry about. I know, it's okay. Oh boy. Okay, yes. Anyways, we need to check out Thanyoko eventually. To get our way over to where we are now currently heading. To head up the eastern coast of this continent and where we know one of the heavenly robots rests. If it wasn't for that promise, I can honestly say I probably wouldn't want to come anywhere near this continent. I can't really blame you. And knowing that we're going up against one of, 3, null, length)))"
"List(The only thing, Ask not what, Every litter, I am)",n

# Python Embedding Models

In [0]:
import os
import openai
from dotenv import load_dotenv
import numpy as np
import pandas as pd
pd.options.display.max_columns = 500

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

df = spark.createDataFrame([("Hello my name is",),("The best code is code thats",),("SynapseML is ",)]).toDF("prompt").toPandas()
df["vector"] = np.nan

for idx, title in zip(df.index.values, df['prompt'].loc[df.index.values]):

  prompt = title
  try:
    results = openai.Embedding.create(input=prompt, deployment_id='text-embedding-ada-002')
    df['vector'].loc[idx] = np.array(results['data'][0]['embedding'], dtype=object)
  except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
display(df)

<command-1088354117486000>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vector'].loc[idx] = np.array(results['data'][0]['embedding'], dtype=object)


prompt vector Hello my name is List(-0.02639627456665039, -0.007323894649744034, -0.027228249236941338, -0.02502225711941719, -0.023055771365761757, 0.04659055918455124, -0.018958929926156998, -0.006154718808829784, -8.579734712839127E-4, -0.015492369420826435, 0.01996738277375698, 0.005152568221092224, -0.0052218991331756115, -0.018971534445881844, -0.013563701882958412, -0.022047318518161774, 0.028740929439663887, 0.00881766714155674, 0.008218897506594658, 0.01197538711130619, -0.021114498376846313, 0.006624280009418726, 0.01362673006951809, 0.002514831256121397, -0.010891299694776535, -0.015139411203563213, 0.020811961963772774, -0.01782441884279251, 0.0166142750531435, -0.01747146062552929, -0.006756639573723078, -0.0019586060661822557, -0.00287409289740026, -0.03706067427992821, 7.094629108905792E-4, -0.034161366522312164, -0.010683306492865086, -0.009069779887795448, 0.003529587760567665, -0.013336800038814545, 0.0182656180113554, 0.005612675100564957, 0.005076934117823839, -0.027379516512155533, -0.01210144441574812, 0.01289560180157423, 0.005511829629540443, -0.010891299694776535, -0.01584532856941223, 0.023837324231863022, 0.025715569034218788, 0.008174777962267399, -0.021190132945775986, -0.016135258600115776, -0.006176778580993414, 0.012750635854899883, -0.007475162856280804, 0.017647938802838326, 3.24596039718017E-4, -0.022425489500164986, 0.02296753227710724, 0.008603370748460293, -0.008332348428666592, 0.013160320930182934, -0.00736171193420887, 0.00721674645319581, 0.00837646797299385, -0.004456104710698128, -0.013387222774326801, 0.014181380160152912, 0.016198286786675453, 0.014925114810466766, -4.057450496475212E-5, -0.0069646332412958145, 0.024026408791542053, 0.00819998886436224, -0.00506117707118392, 0.01705547235906124, -0.004913060460239649, -0.017559699714183807, 0.023471759632229805, -0.035472359508275986, -0.006161021534353495, 0.029346002265810966, 0.03040487878024578, 0.0031057221349328756, -0.01628652773797512, 0.025702962651848793, -0.02497183345258236, -0.025778597220778465, 0.012385072186589241, 0.020206891000270844, -0.009536189958453178, 0.028766140341758728, -0.0370858833193779, 0.0010352407116442919, -0.024165071547031403, 0.02765684202313423, -0.003088389290496707, -0.006926816422492266, -1.1167836782988161E-4, -0.007090690080076456, -0.005477163940668106, -0.006933119148015976, -0.009214745834469795, 0.021467458456754684, -0.018656393513083458, 0.00151583191473037, 0.01045010145753622, -0.007947875186800957, 0.014521732926368713, 0.03179150074720383, 0.011061476543545723, -0.03885067626833916, -0.005867939908057451, -0.0013196561485528946, -0.011445948854088783, -0.010380770079791546, -0.015668848529458046, -0.012807361781597137, -0.0028945771045982838, 0.029900651425123215, -0.003822669619694352, -0.01898414082825184, 0.012448100373148918, 0.009737880900502205, -0.02408943697810173, -0.019274070858955383, 0.007065478712320328, 0.0033720170613378286, 0.025992892682552338, -0.008962632156908512, -0.0019365461776033044, -0.005886848550289869, -0.021568303927779198, 0.01337461732327938, -0.010204290971159935, 0.015593214891850948, -0.0166142750531435, -0.010311438702046871, -0.023358307778835297, 0.03340502828359604, 0.021984290331602097, -0.0037848528008908033, 0.011042567901313305, 0.006832273676991463, -0.011792604811489582, -0.015303284861147404, 0.017509277909994125, 8.33550002425909E-4, 0.0013322618324309587, 0.020395975559949875, 0.019790902733802795, -0.005133659578859806, 0.016904205083847046, 0.03005191870033741, -0.005990845151245594, 0.024291127920150757, -0.010815666057169437, -0.002068905858322978, -0.002445500111207366, 0.00605072220787406, 0.02702655829489231, -0.020471610128879547, -0.013979689218103886, 0.04296012595295906, 0.0019034562865272164, -0.006756639573723078, -0.016929415985941887, 0.006548645906150341, -0.011389223858714104, 0.02054724283516407, -0.01980350911617279, 0.010361861437559128, -0.02166914753615856, -0.011918662115931511, 0.004850031808018684, 0.01144594

# Pyspark Embedding Models

In [0]:
from synapse.ml.cognitive import OpenAIEmbedding

df = spark.createDataFrame([("Hello my name is",),("The best code is code thats",),("SynapseML is ",)]).toDF("prompt")

embedding = (OpenAIEmbedding()
    .setSubscriptionKey(key)
    .setDeploymentName(deployment_name_embeddings)
    .setCustomServiceName(service_name)
    .setTextCol("prompt")
    .setErrorCol("error")
    .setOutputCol("embeddings"))

display(embedding.transform(df))

prompt error embeddings Hello my name is null Map(vectorType -> dense, length -> 1536, values -> List(-0.026396275, -0.0073238946, -0.02722825, -0.025022257, -0.023055771, 0.04659056, -0.01895893, -0.006154719, -8.579735E-4, -0.015492369, 0.019967383, 0.005152568, -0.005221899, -0.018971534, -0.013563702, -0.022047319, 0.02874093, 0.008817667, 0.0082188975, 0.011975387, -0.021114498, 0.00662428, 0.01362673, 0.0025148313, -0.0108913, -0.015139411, 0.020811962, -0.017824419, 0.016614275, -0.01747146, -0.0067566396, -0.001958606, -0.002874093, -0.037060674, 7.094629E-4, -0.034161367, -0.0106833065, -0.00906978, 0.0035295878, -0.0133368, 0.018265618, 0.005612675, 0.005076934, -0.027379517, -0.012101444, 0.012895602, 0.0055118296, -0.0108913, -0.015845329, 0.023837324, 0.025715569, 0.008174778, -0.021190133, -0.016135259, -0.0061767786, 0.012750636, -0.007475163, 0.017647939, 3.2459604E-4, -0.02242549, 0.022967532, 0.008603371, -0.008332348, 0.013160321, -0.007361712, 0.0072167465, 0.008376468, -0.0044561047, -0.013387223, 0.01418138, 0.016198287, 0.014925115, -4.0574505E-5, -0.0069646332, 0.024026409, 0.008199989, -0.005061177, 0.017055472, -0.0049130605, -0.0175597, 0.02347176, -0.03547236, -0.0061610215, 0.029346002, 0.030404879, 0.0031057221, -0.016286528, 0.025702963, -0.024971833, -0.025778597, 0.012385072, 0.020206891, -0.00953619, 0.02876614, -0.037085883, 0.0010352407, -0.024165072, 0.027656842, -0.0030883893, -0.0069268164, -1.1167837E-4, -0.00709069, -0.005477164, -0.006933119, -0.009214746, 0.021467458, -0.018656394, 0.0015158319, 0.010450101, -0.007947875, 0.014521733, 0.0317915, 0.011061477, -0.038850676, -0.00586794, -0.0013196561, -0.011445949, -0.01038077, -0.015668849, -0.012807362, -0.002894577, 0.029900651, -0.0038226696, -0.01898414, 0.0124481, 0.009737881, -0.024089437, -0.01927407, 0.0070654787, 0.003372017, 0.025992893, -0.008962632, -0.0019365462, -0.0058868486, -0.021568304, 0.013374617, -0.010204291, 0.015593215, -0.016614275, -0.010311439, -0.023358308, 0.03340503, 0.02198429, -0.0037848528, 0.011042568, 0.0068322737, -0.011792605, -0.015303285, 0.017509278, 8.3355E-4, 0.0013322618, 0.020395976, 0.019790903, -0.0051336596, 0.016904205, 0.030051919, -0.005990845, 0.024291128, -0.010815666, -0.0020689059, -0.0024455, 0.006050722, 0.027026558, -0.02047161, -0.013979689, 0.042960126, 0.0019034563, -0.0067566396, -0.016929416, 0.006548646, -0.011389224, 0.020547243, -0.01980351, 0.010361861, -0.021669148, -0.011918662, 0.004850032, 0.011445949, -0.032396574, -0.011653943, -0.018668998, -0.003832124, 0.007910059, 0.0030946922, 0.009063478, 0.017622728, 0.028866986, 0.005628432, 0.023660844, -0.030783048, -0.0022233252, 0.03887589, 0.022891898, -0.0018703664, -0.67969775, -0.029774595, 0.024505425, 0.021366613, 0.019034563, 0.015000748, 0.010374467, 0.01371497, -0.008830273, 0.024883594, -0.02547606, 0.040136456, -0.008099143, 0.013298983, -2.3478061E-4, -0.0044718618, 0.004330048, -0.020332947, 0.003209719, 0.013135109, 3.4882256E-4, 0.00789115, -0.012328346, 0.017660545, 0.014572156, 0.020408582, 0.021089287, -0.018820267, -0.017988293, 0.03476644, -0.022299431, 0.023345703, 0.0028425788, -8.839727E-4, 0.05657425, 8.7215484E-4, 0.014635184, 0.03234615, 0.0036871587, 0.03060657, -0.016727725, -0.007777699, -0.0069898446, -0.010922814, 0.012498523, 0.011118202, -0.0021177528, 0.002949727, -9.4542534E-5, 0.0057040663, 0.0024376216, -0.007897452, -0.0062555643, -0.0032365061, 0.0059876936, -0.004865789, 0.0052754735, -0.003318443, 0.004872092, -0.003643039, 0.006983542, 0.015000748, -0.027354306, 0.015832722, -0.0034886193, 0.0035264364, -0.018152166, -0.02018168, 0.006447801, 0.0029245156, 1.6328284E-4, 0.0025825868, -0.004746035, -0.0070213587, 0.0079856925, 0.014811663, 0.003447651, -0.014660396, 9.2966825E-4, 0.004909909, 0.018744633, 0.009971086, -0.012240106, -0.025198737, 0.018820267, -0.032573055, -0.020395976, -8.847605E-4, -0.009139111, 0.0069457246, 0.02818628, 0.009271471, 0.0026172525, -0.031690657, 